In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1,
              0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    return (x - x.mean()) / x.std()

## 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [128]:
def calc_logloss(y, y_pred):
    # хромает - надо чуть дольше копаться, но времени не хватает :(
    '''err = []
    for i, y1 in enumerate(y):
        err.append(- np.mean(y1 * np.log(y_pred[i]) + (1.0 - y1) * np.log(1.0 - y_pred[i])) if y_pred[i] != 0 and y_pred[i] != 1 else np.inf)
    '''
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return np.sum(err)

In [28]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [102]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    return W

In [129]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

## 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [141]:
# на самом деле можно его минимизировать и дальше, задирая alpha, но уже незначительно
W = eval_model(X_st, y, iterations=6000, alpha=4.1)

600 [-28.74032255  -3.09787171  -7.42624192  23.89384414] 0.14017661806098056
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.33812897 0.02344406 0.99999992 0.099703   0.92360624 0.10389955
 1.         0.02305819 0.52318324 0.99999618]
1200 [-33.67944813  -3.40019524  -9.01023912  27.62508239] 0.12301272373068424
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34791475 0.0128215  0.99999999 0.07684741 0.92712411 0.05722096
 1.         0.02079644 0.57460377 0.99999939]
1800 [-37.56040291  -3.73959139 -10.12496892  30.74438703] 0.1123477403916556
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34586395 0.00806038 1.         0.061515   0.94218287 0.04064673
 1.         0.01589979 0.60026949 0.99999988]
2400 [-40.85887     -4.02439489 -11.0717706   33.38421578] 0.10467845103066262
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34226277 0.00538216 1.         0.05039051 0.95213325 0.029935
 1.         0.01269339 0.61970207 0.99999997]
3000 [-43.75432128  -4.27054149 -11.90400342  35.69124072] 0.09879203081058247
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1

In [142]:
probs = np.dot(W, X_st)
probs

array([-0.78246448, -7.03075862, 30.7252566 , -3.90661155,  3.68406142,
       -4.98274002, 36.26791097, -5.21365276,  0.77960906, 22.91488892])

In [146]:
[np.round(z) for z in sigmoid(probs)], y

([0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0],
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

## 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [149]:
def calc_pred_proba(w, x):
    return np.dot(w, x)

In [150]:
calc_pred_proba(W, X_st)

array([-0.78246448, -7.03075862, 30.7252566 , -3.90661155,  3.68406142,
       -4.98274002, 36.26791097, -5.21365276,  0.77960906, 22.91488892])

## 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [157]:
def calc_pred(w, x, threshold=0.5):
    probs = calc_pred_proba(w, x)
    return [1.0 if z >= threshold else 0 for z in sigmoid(probs)]

In [160]:
y_pred = calc_pred(W, X_st)
y_pred

[0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 1.0, 1.0]

## 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [163]:
accuracy = 1/len(y) * np.sum([1 if y[i] == y_pred[i] else 0 for i in range(len(y))])
accuracy

1.0

In [211]:
def confusion_matrix(y, y_pred):
    # делаем пустую матрицу NxN
    uy = list(reversed(np.unique(y)))
    m = [[0 for x in range(len(uy))] for x in range(len(uy))]
    
    for i, yi in enumerate(y):
        m[uy.index(yi)][uy.index(y_pred[i])] += 1
        
    return m

In [222]:
cm = confusion_matrix(y, y_pred)
cm

[[5, 0], [0, 5]]

In [227]:
precision = cm[0][0] / (cm[0][0] + cm[0][1])
precision

1.0

In [228]:
recall = cm[0][0] / (cm[0][0] + cm[1][0])
recall

1.0

In [234]:
beta = 1
f = (1 + beta**2) * ((precision*recall)/(beta**2 * precision+recall)) # полная формула F (просто для проверки)

f1 = 2 * ((precision*recall)/(precision+recall)) # F1
f1

1.0

## 6. Могла ли модель переобучиться? Почему?

Могла. Мы подогнали параметры очень точно для указанных данных, чтобы минимизровать ошибку - в итоге модель возвращает 100% правильный ответ для исходных данных.
Опять же precision и recall равны 1, что указывает на переобущение.

## 7. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно

In [236]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    return W

In [237]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    return W

In [245]:
W_l1 = eval_model_l1(X_st, y, iterations=6000, alpha=4.1, lambda_=1e-4)

600 [-28.40581855  -2.94587529  -7.47485695  23.38930331] 0.14120422470161947
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.3452575  0.02362351 0.99999988 0.1014899  0.9015299  0.08973092
 0.99999999 0.02943766 0.53256934 0.9999944 ]
1200 [-33.17501829  -3.35510371  -8.86476239  27.21776566] 0.12455263814416413
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34784535 0.01359896 0.99999999 0.07897929 0.92490435 0.05967378
 1.         0.02164554 0.57086049 0.99999924]
1800 [-36.85914193  -3.6775415   -9.92336574  30.18039385] 0.11413253487967345
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34627704 0.00876868 1.         0.06406769 0.93979143 0.04323673
 1.         0.0167988  0.59571318 0.99999984]
2400 [-39.96645375  -3.94642835 -10.81530666  32.66939847] 0.10664454776454654
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34323857 0.00600467 1.         0.05319923 0.94970322 0.03250461
 1.         0.01359768 0.61447915 0.99999996]
3000 [-42.67231631  -4.1773759  -11.59266578  34.82812898] 0.1009106099881663
[0. 0. 1. 0. 1. 0. 1. 0. 1.

In [246]:
y_pred_l1 = calc_pred(W_l1, X_st)
y_pred_l1

[0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 1.0, 1.0]

In [247]:
confusion_matrix(y, y_pred_l1)

[[5, 0], [0, 5]]

In [248]:
W_l2 = eval_model_l1(X_st, y, iterations=6000, alpha=4.1, lambda_=1e-4)

600 [-28.40581855  -2.94587529  -7.47485695  23.38930331] 0.14120422470161947
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.3452575  0.02362351 0.99999988 0.1014899  0.9015299  0.08973092
 0.99999999 0.02943766 0.53256934 0.9999944 ]
1200 [-33.17501829  -3.35510371  -8.86476239  27.21776566] 0.12455263814416413
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34784535 0.01359896 0.99999999 0.07897929 0.92490435 0.05967378
 1.         0.02164554 0.57086049 0.99999924]
1800 [-36.85914193  -3.6775415   -9.92336574  30.18039385] 0.11413253487967345
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34627704 0.00876868 1.         0.06406769 0.93979143 0.04323673
 1.         0.0167988  0.59571318 0.99999984]
2400 [-39.96645375  -3.94642835 -10.81530666  32.66939847] 0.10664454776454654
[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.] [0.34323857 0.00600467 1.         0.05319923 0.94970322 0.03250461
 1.         0.01359768 0.61447915 0.99999996]
3000 [-42.67231631  -4.1773759  -11.59266578  34.82812898] 0.1009106099881663
[0. 0. 1. 0. 1. 0. 1. 0. 1.

In [249]:
y_pred_l2 = calc_pred(W_l2, X_st)
y_pred_l2

[0, 0, 1.0, 0, 1.0, 0, 1.0, 0, 1.0, 1.0]

In [250]:
confusion_matrix(y, y_pred_l2)

[[5, 0], [0, 5]]